#In this homework assignment, we apply Multiple Linear Regression (MLR) to a real, clean dataset.

In [1]:
# #@title Mounting your Google Drive
# from google.colab import drive
# drive.mount('/content/drive/', force_remount=True)

In [2]:
#@title Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [3]:
# Import the dataset (name it 'dataset') you have uplouaded into your Google Drive
# Hint: use pandas AND read_csv

dataset = pd.read_csv('50_Startups.csv')

In [4]:
# Run this cell to see the first five rows of the dataset
# Note: if you want to see more rows, use DS.head(number of rows)
dataset.head()

,R&D Spend,Administration,Marketing Spend,State,Profit
0,165349.20,136897.80,471784.10,New York,192261.83
1,162597.70,151377.59,443898.53,California,191792.06
2,153441.51,101145.55,407934.54,Florida,191050.39
3,144372.41,118671.85,383199.62,New York,182901.99
4,142107.34,91391.77,366168.42,Florida,166187.94


In [5]:
# Check if there is any null values in the dataset DS
# Hint: use isna() command
# Expected output is shown is the text cell below.

dataset.isna().sum()

R&D Spend          0
Administration     0
Marketing Spend    0
State              0
Profit             0
dtype: int64

In [6]:
# Run this cell to see dataset information
# Expected output is shown is the text cell below.
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   R&D Spend        50 non-null     float64
 1   Administration   50 non-null     float64
 2   Marketing Spend  50 non-null     float64
 3   State            50 non-null     object 
 4   Profit           50 non-null     float64
dtypes: float64(4), object(1)
memory usage: 2.1+ KB


In [7]:
# Select feature columns (feature matrix) by selecting all columns except the 'Profit'.
# Name them 'X' as it would be a matrix
# Hint: you can select columns by their names or drop the column that you don't want.

X = dataset.drop('Profit', axis=1)

X.head()

,R&D Spend,Administration,Marketing Spend,State
0,165349.20,136897.80,471784.10,New York
1,162597.70,151377.59,443898.53,California
2,153441.51,101145.55,407934.54,Florida
3,144372.41,118671.85,383199.62,New York
4,142107.34,91391.77,366168.42,Florida


In [8]:
# Select the column that you want to predict, i.e., 'Profit'
# Name it 'y' as it would be a vector.

y = dataset['Profit']

y.head()

0    192261.83
1    191792.06
2    191050.39
3    182901.99
4    166187.94
Name: Profit, dtype: float64

In [9]:
# You have to code catgorical data. To do so, use the following libraries.
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

# First step: select dataset column that you want to code.
# Hint: we want to code 'State' column
categorical_feature = ['State']

# Second step: defining the encoder
one_hot = OneHotEncoder()

# Third step: Define transformer to apply encoder, i.e., 'one_hot' to the selected dataset columns, i.e., 'categorical_feature'.
# Hint: use 'ColumnTransformer' command
transformer = ColumnTransformer(transformers=[('one_hot', one_hot, categorical_feature)], remainder='passthrough')

# Fourth step: Fit the defined transformer to the feature matrix X.
# Hint: use 'fit_transform' command
transformed_X = transformer.fit_transform(X)

In [10]:
# Run this cell to see the first five rows of transformed_X
pd.DataFrame(transformed_X).head()

,0,1,2,3,4,5
0,0.0,0.0,1.0,165349.20,136897.80,471784.10
1,1.0,0.0,0.0,162597.70,151377.59,443898.53
2,0.0,1.0,0.0,153441.51,101145.55,407934.54
3,0.0,0.0,1.0,144372.41,118671.85,383199.62
4,0.0,1.0,0.0,142107.34,91391.77,366168.42


In [11]:
# Split the dataset into the Training and Test sets using the imported library;
# Consider test size to be 0.25 and random state to be 2509
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(transformed_X, y, test_size=0.25, random_state=2509)

In [12]:
# This cell standardized training and test data

# Initialize the scaler
scaler = StandardScaler()

# Fit on training set only
scaler.fit(X_train)

# Apply transform to both the training set and the test set
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Training the Multiple Linear Regression model on the Training set

In [13]:
# Train the Multiple Linear Regression model on the Training set

# First step: import linear regression from sklearn
from sklearn.linear_model import LinearRegression

# Second step: define regeressor using the imported linear regression
regressor = LinearRegression()

# Fit the defined regeressor to the traniing dataset, including X_train_scaled and y_train
regressor.fit(X_train_scaled, y_train)

LinearRegression()

In [14]:
# Evaulate the performance of your model on X_test_scaled and y_test.
# In the context of multilinear regression),
# Hint: use .score() method to return the coefficient of determination, also known as R² (R-squared).
# We expect output > 0.98
regressor.score(X_test_scaled, y_test)

0.9840064291741724

In [18]:
# Apply cross validation (CV)

# First step: import CV score from sklearn
from sklearn.model_selection import cross_val_score

# Second step: Apply K-Fold cross-validation
# Hint: use the imported library, consider K=5, and use negative mean squared error for scoring
cv_scores = cross_val_score(regressor, X_train_scaled, y_train, cv=5, scoring='neg_mean_squared_error')

# Print the score mean (± standard deviation);
# Hint: use mean() and std() commands
print(f'Score mean: {-cv_scores.mean()} {{± {cv_scores.std()}}}')

Score mean: 139925136.1761989 {± 95111949.50619605}


# Predicting the Test set results

In [19]:
# Apply the regressor on X_test_scaled to predict y_pred
y_pred = regressor.predict(X_test_scaled)

In [20]:
# This cell shows the actual label vector y_test and the predicted label vector y_pred
d = {'y_pred': y_pred, 'y_test': y_test}
pd.DataFrame(d)

,y_pred,y_test
32,98884.371543,97427.84
33,100047.235184,96778.92
47,47766.247901,42559.73
9,154976.558305,149759.96
37,91129.087779,89949.14
8,151755.926389,152211.77
23,112436.195860,108733.99
24,113375.898676,108552.04
17,130706.106786,125370.37
1,189141.730655,191792.06


In [21]:
# As an example metric, run this cell to determine Mean Squared Error
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

Mean Squared Error: 24637182.386300746


## **(2) Ridge Regression**

In [22]:
# Apply the Ridge Regression

# First step: import Ridge from sklearn
from sklearn.linear_model import LinearRegression, Ridge

# Define ridge_model based on the imported Ridge; consider alpha=1
# alpha is the regularization strength
ridge_model = Ridge(alpha=1)

# Fit the ridge_model to the X_train_scaled and y_train
ridge_model.fit(X_train_scaled, y_train)

Ridge(alpha=1)

In [23]:
# Use Ridge Regression for prediction
# Hint: Apply ridge_model to X_test_scaled
y_pred_ridge = ridge_model.predict(X_test_scaled)

# Determine mean squared error of the ridge_model
# Between y_test and y_pred_ridge
mse_ridge = mean_squared_error(y_test, y_pred_ridge)

# print mse_ridge
print("Mean Squared Error for Ridge:", mse_ridge)

Mean Squared Error for Ridge: 38221204.93274074


In [24]:
# Apply K-Fold cross-validation to ridge_model
# Note: You have already imported CV score from sklearn
# consider K=5, and use negative mean squared error for scoring
ridge_cv_scores = cross_val_score(ridge_model, X_train_scaled, y_train, cv=5, scoring='neg_mean_squared_error')

# Print CV MSE for Ridge Regression, i.e., the score mean (± standard deviation);
print(f'Score mean for Ridge: {-ridge_cv_scores.mean()} {{± {ridge_cv_scores.std()}}}')

Score mean for Ridge: 141327893.77362555 {± 96319496.3190408}


## **(3) Lasso Regression**

In [25]:
# Apply the Lasso Regression

# First step: import Lasso from sklearn
from sklearn.linear_model import LinearRegression, Lasso

# Define lasso_model based on the imported Lasso; consider alpha=0.25
# alpha is the regularization strength
lasso_model = Lasso(alpha=0.25)

# Fit the lasso_model to the X_train_scaled and y_train
lasso_model.fit(X_train_scaled, y_train)

Lasso(alpha=0.25)

In [26]:
# Use Lasso Regression for prediction
# Hint: Apply lasso_model to X_test_scaled
y_pred_lasso = lasso_model.predict(X_test_scaled)

# Determine mean squared error of the lasso_model
# Between y_test and y_pred_lasso
mse_lasso = mean_squared_error(y_test, y_pred_lasso)

# print mse_lasso
print("Mean Squared Error for Lasso:", mse_lasso)

Mean Squared Error for Lasso: 24636723.126664046


In [31]:
# Apply K-Fold cross-validation to lasso_model
# Note: You have already imported CV score from sklearn
lasso_cv_scores = cross_val_score(lasso_model, X_train_scaled, y_train, cv=5, scoring='neg_mean_squared_error')

# Print CV MSE for Lasso Regression, i.e., the score mean (± standard deviation);
print(f'Score mean for Lasso: {-lasso_cv_scores.mean()} {{± {lasso_cv_scores.std()}}}')

Score mean for Lasso: 139919852.72097132 {± 95109906.7777273}


/Users/gowda/UMass/EC 601/HW#3 Due Date Feb 27/venv/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.247e+08, tolerance: 5.356e+06
  model = cd_fast.enet_coordinate_descent(
/Users/gowda/UMass/EC 601/HW#3 Due Date Feb 27/venv/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.566e+08, tolerance: 4.870e+06
  model = cd_fast.enet_coordinate_descent(


In [32]:
# Check Coefficients correspond to each feature;
# Larger cofficient shows more impact of that feature on the output

print(f'Coefficients of the original model are: {regressor.coef_}')
print(f'Intercept of the original model is: {regressor.intercept_}')

print(f'\n Coefficients of the Ridge model are: {ridge_model.coef_}')
print(f'Intercept of the Ridge model is: {ridge_model.intercept_}')

print(f'\n Coefficients of Ridge model are: {lasso_model.coef_}')
print(f'Intercept of the Lasso model is: {lasso_model.intercept_}')

Coefficients of the original model are: [  -96.61229619   291.32877769  -190.14891362 35503.50697956
  -753.56588837  4177.06348715]
Intercept of the original model is: 113422.9021621622

 Coefficients of the Ridge model are: [ -277.29482555   453.3715394   -176.27514538 33234.09827798
  -174.087211    5747.77171686]
Intercept of the Ridge model is: 113422.9021621622

 Coefficients of Ridge model are: [   -0.           390.19833983   -87.40541941 35503.19410104
  -753.2265558   4177.11061285]
Intercept of the Lasso model is: 113422.9021621622
